# Drosophila Melanogaster

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import Bio
import torch as torch
from torch.utils.data import Dataset, DataLoader
print("Biopython v" + Bio.__version__)
from timeit import default_timer as timer
import copy
import math
import sys




Biopython v1.78


In [2]:
from Bio import SeqIO
count = 0
sequences = [] # Here we are setting up an array to save our sequences for the next step

for seq_record in SeqIO.parse("./Genomedata/genome.fa", "fasta"):
    if (count < 6):
        sequences.append(seq_record)
        print("Id: " + seq_record.id + " \t " + "Length: " + str("{:,d}".format(len(seq_record))) )
        print(repr(seq_record.seq) + "\n")
        count = count + 1

sequences.pop()

Id: chr2L 	 Length: 23,513,712
Seq('Cgacaatgcacgacagaggaagcagaacagatatttagattgcctctcattttc...gag')

Id: chr2R 	 Length: 25,286,936
Seq('CTCAAGATAccttctacagattatttaaagctagtgcacaacaacaataaattg...ttc')

Id: chr3L 	 Length: 28,110,227
Seq('TAGGGAGAAATATGATCgcgtatgcgagagtagtgccaacatattgtgctcttt...tat')

Id: chr3R 	 Length: 32,079,331
Seq('acgggaccgagtatagtaccagtacgcgaccagtacgggagcagtacggaacca...ttc')

Id: chr4 	 Length: 1,348,131
Seq('ttattatattattatattattatattattatattattatattattatattatta...GAA')

Id: chrM 	 Length: 19,524
Seq('AATGAATTGCCTGATAAAAAGGATTACCTTGATAGGGTAAATCATGCAGTTTTC...ATT')



SeqRecord(seq=Seq('AATGAATTGCCTGATAAAAAGGATTACCTTGATAGGGTAAATCATGCAGTTTTC...ATT'), id='chrM', name='chrM', description='chrM', dbxrefs=[])

In [3]:
sequences

[SeqRecord(seq=Seq('Cgacaatgcacgacagaggaagcagaacagatatttagattgcctctcattttc...gag'), id='chr2L', name='chr2L', description='chr2L', dbxrefs=[]),
 SeqRecord(seq=Seq('CTCAAGATAccttctacagattatttaaagctagtgcacaacaacaataaattg...ttc'), id='chr2R', name='chr2R', description='chr2R', dbxrefs=[]),
 SeqRecord(seq=Seq('TAGGGAGAAATATGATCgcgtatgcgagagtagtgccaacatattgtgctcttt...tat'), id='chr3L', name='chr3L', description='chr3L', dbxrefs=[]),
 SeqRecord(seq=Seq('acgggaccgagtatagtaccagtacgcgaccagtacgggagcagtacggaacca...ttc'), id='chr3R', name='chr3R', description='chr3R', dbxrefs=[]),
 SeqRecord(seq=Seq('ttattatattattatattattatattattatattattatattattatattatta...GAA'), id='chr4', name='chr4', description='chr4', dbxrefs=[])]

In [4]:
agsts = pd.read_csv('./Genomedata/genes-augustus.csv')

In [5]:
agsts['chrom'].unique()

array(['chr2L', 'chr2R', 'chr3L', 'chr3R', 'chr4', 'chrUn_CP007080v1',
       'chrUn_CP007081v1', 'chrUn_CP007097v1', 'chrUn_DS483755v1',
       'chrUn_DS483780v1', 'chrUn_DS483783v1', 'chrUn_DS483801v1',
       'chrUn_DS483878v1', 'chrUn_DS484006v1', 'chrUn_DS484105v1',
       'chrUn_DS484134v1', 'chrUn_DS484375v1', 'chrUn_DS484521v1',
       'chrUn_DS484581v1', 'chrUn_DS484779v1', 'chrUn_DS485083v1',
       'chrUn_DS485107v1', 'chrUn_DS485610v1', 'chrX',
       'chrX_DS483995v1_random', 'chrY', 'chrY_CP007109v1_random',
       'chrY_CP007113v1_random', 'chrY_CP007116v1_random',
       'chrY_DS483742v1_random', 'chrY_DS483778v1_random',
       'chrY_DS483875v1_random', 'chrY_DS483931v1_random',
       'chrY_DS484390v1_random', 'chrY_DS484643v1_random',
       'chrY_DS485048v1_random', 'chrY_DS485159v1_random'], dtype=object)

Finding the sequences 

In [6]:
def code_Exon_Intron(coding_row, x, y):
    ch[coding_row['txStart']:coding_row['txEnd']] = 'T'
    Estart = np.array(coding_row['exonStarts'].split(',')[:-1]).astype(int)
    Eend = np.array(coding_row['exonEnds'].split(',')[:-1]).astype(int)
    y_star = y[Estart[0]:Eend[-1]]
    x_star = x[Estart[0]:Eend[-1]]
    for i in range(len(Estart)):
        y_star[Estart[i]:Eend[i]+1] = 'E1'
        if i != len(Estart) - 1:
            y_star[Eend[i]+1:Estart[i+1]] = 'I'
    return ch

def code_sequence(augustus, sequences):
    df = pd.DataFrame(columns=['id','x','y'])
    
    for seq in sequences:
        s_id_aug = augustus.loc[augustus['chrom'] == seq.id]

        y = np.array(['N'] * s_id_aug.iloc[-1]['txEnd'],dtype='U')

        t = np.array([rec for rec in str(seq.seq)],dtype='U')[:s_id_aug.iloc[-1]['txEnd']]
        x = np.array([x.upper() if isinstance(x, str) else x for x in t])


        for index, row in s_id_aug.iterrows():

            if row['name'][-3:] == '.t2':
                y = code_Exon_Intron(row, x, y)
                
            check_t2 = row['name'][:-3] + '.t2'
            if s_id_aug[s_id_aug['name'].isin([check_t2])].empty:
                continue
            else: 
                y = code_Exon_Intron(row, y)
        # print(x[0],'\n',x[1],'\n/',x[2],'\n',x[3],'\n',x[100])
        print(len(x.tolist()))
        char = pd.Series({'id': seq.id, 'x':x.tolist(), 'y':y.tolist()})
        print(char['x'][:20])
        df = df.append(char,ignore_index=True)
    return df

In [7]:
temp = code_sequence(agsts, sequences)


23371726
['C', 'G', 'A', 'C', 'A', 'A', 'T', 'G', 'C', 'A', 'C', 'G', 'A', 'C', 'A', 'G', 'A', 'G', 'G', 'A']
25254583
['C', 'T', 'C', 'A', 'A', 'G', 'A', 'T', 'A', 'C', 'C', 'T', 'T', 'C', 'T', 'A', 'C', 'A', 'G', 'A']
27866355
['T', 'A', 'G', 'G', 'G', 'A', 'G', 'A', 'A', 'A', 'T', 'A', 'T', 'G', 'A', 'T', 'C', 'G', 'C', 'G']
32070586
['A', 'C', 'G', 'G', 'G', 'A', 'C', 'C', 'G', 'A', 'G', 'T', 'A', 'T', 'A', 'G', 'T', 'A', 'C', 'C']
1271726
['T', 'T', 'A', 'T', 'T', 'A', 'T', 'A', 'T', 'T', 'A', 'T', 'T', 'A', 'T', 'A', 'T', 'T', 'A', 'T']


In [9]:
def prepare_segment(x,y):
    df = pd.DataFrame(columns = ['x','y'])
    last_i = 0
    for i in range(1000,len(x),1000):
        df = df.append({'x': x[last_i:i], 'y':['START'] + y[last_i:i] + ['STOP']}, ignore_index = True)
        last_i = i
    return df

In [10]:
df = prepare_segment(temp.iloc[4]['x'],temp.iloc[4]['y'])

In [79]:
class MultiDimnDict():
    
    def __init__(self,axes):
        self.axes = axes

        self.data = {}
        prev = None

        for i in range(len(axes)-1, -1, -1):
            x = {}
            for n in axes[i]:
                if prev == None:
                    x[n] = 0
                else:
                    x[n] = copy.deepcopy(prev)
            prev = x
            self.data = x

    def __get_string(self, x,_str, depth):

        if type(list(x.values())[0]) == type(dict()):
            
            for i in x:

                _str += self.__get_string(x[i], depth * " " + i + "  \n", depth+1)
                # print(_str)
            return _str
        else:
            return _str + " " * depth +str(x) +"\n"

    def __str__(self):
        string = self.__get_string(self.data, "", 0)
        return string

    def add_value_helper(self,lst,val, temp, depth):
        label = lst[0]
        if len(lst) == 1:
            if label == ':':
                for x in self.axes[depth]:
                    temp[x] += val
            else:
                temp[label] += val
            return temp
        
        # print(lst[1:],val,temp[lst[0]])
        if label == ':':
            for x in self.axes[depth]:
                temp[x] = self.add_value_helper(lst[1:], val, temp[x], depth+1)
        else:
            temp[label] = self.add_value_helper(lst[1:],val,temp[label], depth+1)
        return temp


    def add_value(self,lst,val):
        if len(lst) == len(self.axes):
            self.data =  self.add_value_helper(lst,val,self.data, 0)
            return self
        else:
            return 0

    def to_tensor(self):
        key_x = {}
        key_x[i] = val for i,val in enumerate(key_x)
        key_y = {}
        key_y[i] = val for i,val in enumerate(key_y)

        
    
    

x= MultiDimnDict([['A', 'N', 'G', 'C', 'T'], ['E', 'START', 'STOP', 'N', 'I'], ['E', 'START', 'STOP', 'N', 'I']])

print(x.add_value(['A','N','N'], 1))


A  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 1, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
N  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
G  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
C  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'STAR

Cant write to dataframe since the character array turns into shit

Steps

1. Create the count matrices
2. Use viterbi algo

In [93]:
class HMM():
    def __init__(self,data,num_x,num_y,vocab_x,vocab_y):
        self.data = data
        self.vocab_x = vocab_x
        self.vocab_y = vocab_y
        self.num_x = num_x
        self.num_y = num_y
    
    def helper(self, c1,c2,c3, vocab):
        if len(vocab) == 1:
            for i in vocab[0]:
                if c3[i] > 0:
                    c1[i] = c2[i] / c3[i]
                else:
                    c1[i] = 0
            return c1
        else:
            for i in vocab[0]:
                c1[i] = self.helper(c1[i], c2[i], c3, vocab[1:])
            return c1


    def calculate_emissions(self):

        axes_for_emissions = []
        for i in range(self.num_x):
            axes_for_emissions.append('x')
        for i in range(self.num_y):
            axes_for_emissions.append('y')
        vocab_for_emissions = []
        for axe in axes_for_emissions:
            if axe == 'y':
                vocab_for_emissions.append(list(self.vocab_y))
            elif axe == 'x':
                vocab_for_emissions.append(list(self.vocab_x))
        self.emissions = MultiDimnDict(vocab_for_emissions)
        count_emit = MultiDimnDict(vocab_for_emissions)
        count_yi = MultiDimnDict([list(self.vocab_y)])

        for i, row in self.data.iterrows():
            _iter = 1
            x = row['x']
            y = row['y']

            while _iter < len(y) - 2:
                _data = []
                list_y = []
                for i in range(self.num_y):
                    if (_iter - i) >= 0:
                        list_y.append(y[_iter - i])
                    else:
                        list_y.append(':')
                list_y = list_y[::-1] # so that the list is []....,y-2, y-1, y]
                list_x = []
                for i in range(self.num_x):
                    if (_iter - i - 1) >= 0:
                        list_x.append(x[_iter - i - 1])       
                    else:
                        list_x.append(':')
                list_x = list_x[::-1]
                _data = list_x + list_y
                count_emit.add_value(_data,1)
                count_yi.add_value([list_y[-1]],1)

                _iter += 1
            


        

            
        self.emissions.data = self.helper(self.emissions.data, count_emit.data, count_yi.data, vocab_for_emissions)
        return self.emissions




    def viterbi(self, test_x):
        ## Final Goal: Find PI(n,v) where n is the length of y without start and stop
        ## We wont use PI we will maximize negative log pi
        ## Step 1: Find PI(1,v)

        k = 1
        viterbi_y = ['START']
        PI=[]
        _pi = []
        while k < len(test_x)+1:
            list_x = []
            list_y = []
            for i in range(self.num_x):
                if (k - i - 1) >= 0:
                    list_x.append(test_x[k - i - 1])       
                else:
                    list_x.append(':')
            list_x = list_x[::-1]
            for i in range(self.num_y):
                if i == 0:
                    continue
                elif (k-i) >= 0:
                    list_y.append(viterbi_y[k-i])
                else:
                    list_y.append(':')
            list_y = list_y[::-1]

            _pi = []
            for pred_y_k in self.vocab_y:
                _data = list_x + list_y + [pred_y_k]
                a = self.emissions.get_sum(_data)
                nlog_a = sys.maxsize
                if a > pow(10,-7):
                    nlog_a = -1 * math.log(a)
                if len(PI) == 0:
                    _pi.append(nlog_a)
                else: 
                    
                    _pi.append(nlog_a + PI[-1][list(self.vocab_y).index(viterbi_y[-1])])

            min_pi = min(_pi)
            min_y = list(self.vocab_y)[_pi.index(min_pi)]

            viterbi_y.append(min_y)
            PI.append(_pi)

            k+= 1
        print(len(PI), 'Should be 1000')
        print(len(viterbi_y), 'Should be 1001 including Start tag')


        PI = PI[::-1]
        y_star = 'STOP'
        final_Y = []
        final_Y.append(y_star)
        for i in range(self.num_y -1):
            final_Y.append(viterbi_y[len(viterbi_y)-1-i])
            del PI[0]
        for _PI in PI:
            pred_pi = []
            index = PI.index(_PI)
            list_x = []
            list_y = []
            for i in range(self.num_x):
                if (index - i) >= 0:
                    list_x.append(test_x[index - i])       
                else:
                    list_x.append(':')
            list_x = list_x[::-1]
            for i in range(self.num_y-1):
                if (index-i) >= 0:
                    list_y.append(viterbi_y[index-i])
                else:
                    list_y.append(':')
            list_y = list_y[::-1]
            for y in self.vocab_y:
                _data = list_x + [y] + list_y 
                emission = sys.maxsize
                a = self.emissions.get_sum(_data)
                if a > pow(10,-7):
                    emission = math.log(a) * -1
                pred_pi.append(_PI[list(self.vocab_y).index(y)] + emission)


            min_pi = min(pred_pi)
            final_y = list(self.vocab_y)[pred_pi.index(min_pi)]


            final_Y.append(final_y)

        final_Y.append('START')
        final_Y = final_Y[::-1]
        viterbi_y += ['STOP']
        return final_Y, viterbi_y
            

        



In [94]:
hmm = HMM(df,3,6 ,set(temp.iloc[0]['x']), set(temp.iloc[0]['y'] + ['START','STOP']))


In [95]:
start = timer()
hmm.calculate_emissions()
stop = timer()
print(stop-start)

ZeroDivisionError: division by zero

In [26]:
start = timer()
hmm.calculate_transitions()
stop = timer()
print(stop-start)

STOP
STOP
STOP
STOP
STOP
1.5163649559999612


In [81]:
print(hmm.transitions)

E  
 {'E': 0.9876951821700254, 'STOP': 0.009955958894968638, 'START': 0.0, 'N': 0.00013345789403443215, 'I': 0.0022154010409715734}
STOP  
 {'E': 0, 'STOP': 0, 'START': 0, 'N': 0, 'I': 0}
START  
 {'E': 0.02940945191475977, 'STOP': 0.0, 'START': 0.0, 'N': 0.8570417551309271, 'I': 0.11354879295431312}
N  
 {'E': 4.586958177032733e-06, 'STOP': 0.009998651434295952, 'START': 0.0, 'N': 0.989996761607527, 'I': 0.0}
I  
 {'E': 0.0005687019724248898, 'STOP': 0.010021638416511776, 'START': 0.0, 'N': 0.0, 'I': 0.9894096596110633}



In [89]:
print(hmm.emissions)

A  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
N  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
G  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 N  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 I  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
C  
 E  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 START  
  {'E': 0, 'START': 0, 'STOP': 0, 'N': 0, 'I': 0}
 STOP  
  {'E': 0, 'STAR

In [83]:
start = timer()
y_pred, vy = hmm.viterbi(temp.iloc[4]['x'])
stop = timer()
print(stop-start)


Done 1  [1.1577945174954207, 9223372036854775807, 9223372036854775807, 1.1354919087530715, 1.1043114825750735]
PI=  99
0     [3.8283668569803533, 9.223372036854776e+18, 9.223372036854776e+18, 4.080162186559459, 4.126571043154425]
1.424188367993338


In [84]:
print(len(y_pred),'\n',len(df.iloc[4]['y']))
print(y_pred[-1], df.iloc[4]['y'][-1])

102 
 102
STOP STOP


In [85]:
print(y_pred)

['START', 'I', 'E', 'I', 'E', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'E', 'E', 'I', 'I', 'I', 'I', 'E', 'E', 'I', 'I', 'I', 'E', 'E', 'I', 'E', 'E', 'I', 'I', 'E', 'E', 'E', 'I', 'E', 'E', 'I', 'I', 'I', 'I', 'E', 'E', 'E', 'E', 'E', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'E', 'E', 'I', 'I', 'I', 'E', 'I', 'I', 'I', 'E', 'E', 'E', 'E', 'E', 'I', 'I', 'E', 'E', 'E', 'I', 'I', 'E', 'E', 'E', 'E', 'I', 'E', 'I', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'I', 'E', 'E', 'E', 'E', 'E', 'STOP']


In [86]:
count = 0
y_corr = ['START'] + df.iloc[4]['y'] + ['STOP']
for i in range(len(y_pred)):
    if y_pred[i] == y_corr[i]:
        count += 1
print(count/len(y_pred))

0.00980392156862745


In [87]:

print(set(y_corr))
print(set(temp_y))
print(set(y_pred))
print([y_corr.count(i) for i in set(y_corr)])
print([temp_y.count(i) for i in set(temp_y)])
print([y_pred.count(i) for i in set(y_pred)])

{'N', 'STOP', 'START'}
{'I', 'E'}
{'I', 'STOP', 'E', 'START'}
[100, 2, 2]
[56, 43]
[51, 1, 49, 1]


In [90]:
start = timer()
y_pred,temp_y = hmm.viterbi(temp.iloc[4]['x'][:20000])
stop = timer()
print(stop-start)


Done 1  [1.1577945174954207, 9223372036854775807, 9223372036854775807, 1.1354919087530715, 1.1043114825750735]
PI=  19999
15000     [38168.282978398754, 9.223372036854815e+18, 9.223372036854815e+18, 38167.86864575531, 38167.80596789631]
10000     [25408.808828052264, 9.2233720368548e+18, 9.2233720368548e+18, 25408.31594031823, 25408.273514414625]
5000     [12661.542118926282, 9.223372036854788e+18, 9.223372036854788e+18, 12661.637173086183, 12661.646865710436]
0     [3.8283668569803533, 9.223372036854776e+18, 9.223372036854776e+18, 4.080162186559459, 4.126571043154425]
3.2705980879982235


In [95]:
print(len(y_pred),'\n',len(temp.iloc[4]['y'][:20000]))
print(y_pred[-1], temp.iloc[4]['y'][-1])

20002 
 20000
STOP N


In [98]:
count = 0
y_corr = ['START'] + temp.iloc[4]['y'][:20000] + ['STOP']
print(len(y_pred), len(y_corr))
for i in range(len(y_pred)):
    if y_pred[i] == y_corr[i]:
        count += 1
print(count/len(y_pred))

20002 20002
9.999000099990002e-05


In [99]:

print(set(y_corr))
print(set(temp_y))
print(set(y_pred))
print([y_corr.count(i) for i in set(y_corr)])
print([temp_y.count(i) for i in set(temp_y)])
print([y_pred.count(i) for i in set(y_pred)])

{'N', 'STOP', 'START'}
{'I', 'E'}
{'I', 'STOP', 'E', 'START'}
[20000, 1, 1]
[10028, 9971]
[10106, 1, 9894, 1]
